# AE simulations
Comparing results of E+ v9.2 to v9.5 where the incident solar angle bug was fixed.

Annual energy and demand data only.

3 vintages, 6 climate zones (NREL's default cities)

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
import scipy.stats as sts
from scipy.stats import linregress
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [2]:
# # 2018 Egrid Costs (elec $/kWh, gas $/therm), change values per city
# gas = 0.83
# elec = 0.1314

---

## Import all csv files

In [3]:
energy92_path = "data/AllEnergy_92.csv"
energy95_path = "data/AllEnergy_95.csv"
demand92_path = "data/AllDemand_92.csv"
demand95_path = "data/AllDemand_95.csv"
demandAll_path = "data/AllDemand.csv"

In [4]:
demandAll = pd.read_csv(demandAll_path)

In [5]:
demandAll

,Run,Climate,System,Parametric,Heating [kBtuh]_92,Cooling [kBtuh]_92,Fan [kBtuh]_92,Heat [kW]_92,Cool [kW]_92,Fan [kW]_92,...,Heating [kBtuh]_95,Cooling [kBtuh]_95,Fan [kBtuh]_95,Heat [kW]_95,Cool [kW]_95,Fan [kW]_95,Demand [kW]_95,U,SHGC,Demand%Diff
0,1,2A,Elec,HOU+elecres+slab+IECC_1990_para00001Table.csv,30.0,0.00,3.00,8.79,0.0,0.91000,...,30.0,0.00,3.00,8.79,0.0,0.91000,9.70000,0.1,0.10,0.000000
1,2,2A,Elec,HOU+elecres+slab+IECC_1990_para00002Table.csv,30.0,0.00,1.00,8.79,0.0,0.42000,...,30.0,0.00,1.00,8.79,0.0,0.43000,9.22000,0.1,0.25,0.108578
2,3,2A,Elec,HOU+elecres+slab+IECC_1990_para00003Table.csv,30.0,0.00,2.00,8.79,0.0,0.47000,...,30.0,0.00,2.00,8.79,0.0,0.48000,9.27000,0.1,0.40,0.107991
3,4,2A,Elec,HOU+elecres+slab+IECC_1990_para00004Table.csv,30.0,0.00,2.00,8.79,0.0,0.52000,...,30.0,0.00,2.00,8.79,0.0,0.54000,9.33000,0.1,0.55,0.214823
4,5,2A,Elec,HOU+elecres+slab+IECC_1990_para00005Table.csv,30.0,0.00,2.00,8.79,0.0,0.58000,...,30.0,0.00,2.00,8.79,0.0,0.59000,9.38000,0.1,0.70,0.106724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,32,7,Gas,DLH+gasfurnace+slab+IECC_2018_para00032Table.csv,0.0,5.03,1.24,0.00,1.5,0.36332,...,0.0,5.07,1.23,0.00,1.5,0.36039,1.86039,1.1,0.25,-0.157246
1292,33,7,Gas,DLH+gasfurnace+slab+IECC_2018_para00033Table.csv,0.0,5.72,1.45,0.00,1.7,0.42485,...,0.0,5.79,1.46,0.00,1.7,0.42778,2.12778,1.1,0.40,0.137892
1293,34,7,Gas,DLH+gasfurnace+slab+IECC_2018_para00034Table.csv,0.0,6.23,1.61,0.00,1.8,0.47173,...,0.0,6.21,1.61,0.00,1.8,0.47173,2.27173,1.1,0.55,0.000000
1294,35,7,Gas,DLH+gasfurnace+slab+IECC_2018_para00035Table.csv,0.0,7.10,1.87,0.00,2.1,0.54791,...,0.0,7.08,1.87,0.00,2.1,0.54791,2.64791,1.1,0.70,0.000000


## Find the percent difference from 9.2 to 9.5

In [6]:
demandAll["DemandPercentDifference"] = ((demandAll['Demand [kW]_95'] - demandAll['Demand [kW]_92']) / demandAll['Demand [kW]_92']) * 100

demandAll

,Run,Climate,System,Parametric,Heating [kBtuh]_92,Cooling [kBtuh]_92,Fan [kBtuh]_92,Heat [kW]_92,Cool [kW]_92,Fan [kW]_92,...,Cooling [kBtuh]_95,Fan [kBtuh]_95,Heat [kW]_95,Cool [kW]_95,Fan [kW]_95,Demand [kW]_95,U,SHGC,Demand%Diff,DemandPercentDifference
0,1,2A,Elec,HOU+elecres+slab+IECC_1990_para00001Table.csv,30.0,0.00,3.00,8.79,0.0,0.91000,...,0.00,3.00,8.79,0.0,0.91000,9.70000,0.1,0.10,0.000000,0.000000
1,2,2A,Elec,HOU+elecres+slab+IECC_1990_para00002Table.csv,30.0,0.00,1.00,8.79,0.0,0.42000,...,0.00,1.00,8.79,0.0,0.43000,9.22000,0.1,0.25,0.108578,0.108578
2,3,2A,Elec,HOU+elecres+slab+IECC_1990_para00003Table.csv,30.0,0.00,2.00,8.79,0.0,0.47000,...,0.00,2.00,8.79,0.0,0.48000,9.27000,0.1,0.40,0.107991,0.107991
3,4,2A,Elec,HOU+elecres+slab+IECC_1990_para00004Table.csv,30.0,0.00,2.00,8.79,0.0,0.52000,...,0.00,2.00,8.79,0.0,0.54000,9.33000,0.1,0.55,0.214823,0.214823
4,5,2A,Elec,HOU+elecres+slab+IECC_1990_para00005Table.csv,30.0,0.00,2.00,8.79,0.0,0.58000,...,0.00,2.00,8.79,0.0,0.59000,9.38000,0.1,0.70,0.106724,0.106724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,32,7,Gas,DLH+gasfurnace+slab+IECC_2018_para00032Table.csv,0.0,5.03,1.24,0.00,1.5,0.36332,...,5.07,1.23,0.00,1.5,0.36039,1.86039,1.1,0.25,-0.157246,-0.157246
1292,33,7,Gas,DLH+gasfurnace+slab+IECC_2018_para00033Table.csv,0.0,5.72,1.45,0.00,1.7,0.42485,...,5.79,1.46,0.00,1.7,0.42778,2.12778,1.1,0.40,0.137892,0.137892
1293,34,7,Gas,DLH+gasfurnace+slab+IECC_2018_para00034Table.csv,0.0,6.23,1.61,0.00,1.8,0.47173,...,6.21,1.61,0.00,1.8,0.47173,2.27173,1.1,0.55,0.000000,0.000000
1294,35,7,Gas,DLH+gasfurnace+slab+IECC_2018_para00035Table.csv,0.0,7.10,1.87,0.00,2.1,0.54791,...,7.08,1.87,0.00,2.1,0.54791,2.64791,1.1,0.70,0.000000,0.000000


In [7]:
demandAll_group= demandAll.groupby(["Climate", "System"])

demandAll_group.mean()

demandAll_group.mean().to_csv("data/grouped_demand.csv")